In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import os
import PIL
from datetime import datetime
from functools import partial
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras import Model

from skimage import io, color, filters
from skimage.transform import resize, rotate
from sklearn.metrics import confusion_matrix, accuracy_score, recall_score, precision_score, f1_score

In [8]:
# Image data generator & flow from dir
train_path = 'disease_data3/train'
test_path = 'disease_data3/test'
valid_path = 'disease_data3/valid'
bs = 32

In [9]:
train_batches = ImageDataGenerator(rescale=1/255, horizontal_flip=True, vertical_flip=True, shear_range=0.2, rotation_range=60).flow_from_directory(train_path, target_size=(224, 224), batch_size=bs)
test_batches = ImageDataGenerator(rescale=1/255).flow_from_directory(test_path, target_size=(224, 224), batch_size=bs)
valid_batches = ImageDataGenerator(rescale=1/255).flow_from_directory(valid_path, target_size=(224, 224), batch_size=bs)

Found 8747 images belonging to 10 classes.
Found 1148 images belonging to 10 classes.
Found 1208 images belonging to 10 classes.


In [11]:
def image_plot_predict(rows, batch, model, cols=bs):  
    """ 
    Plots images with thier predicted class and actual class  
  
    Used after model has been fit to see results of test data
  
    Parameters: 
    rows (int): number of rows for the images displayed
    batch (ImageDataGenerator): test batch of pictures that will be predicted in model
    model : the model training images have been trained on to be used to predict test images
    cols (int) : number of columns displayed ( set to batch size )
    
    
    Returns: 
    plot: Plot of images with their predicted class and their actual class
    """
    types = ['Apple - Apple Scab', 'Apple - Black Rot', 'Apple - Cedar Apple Rust', 'Apple - Healthy', 'Corn - Cercospora Leaf Spot', 'Corn - Common Rust', 'Corn - Northern Leaf Blight', 'Corn - Healthy', 'Grape - Black Rot', 'Grape - Healthy']
    fig, axs = plt.subplots(rows,cols,figsize=(cols * 4,rows * 4))
    for i in range(rows):
        images, labels = next(batch)
        predictions = model.predict(images)
        for j, pic in enumerate(images):
            title = 'Predicted: ' + types[list(predictions[j]).index(predictions[j].max())] + ' ' + '\n' + 'Actual: ' + types[list(labels[j]).index(1)] + '\n' + 'Confidence: ' + str(predictions[j].max().round(2))
            if rows > 1:
                axs[i,j].imshow(pic, extent=[0, 20, 0, 15], aspect=1)
                axs[i,j].set_title(title)
                axs[i,j].axis('off')
                if types[list(predictions[j]).index(predictions[j].max())] != types[list(labels[j]).index(1)]:
                    axs[i,j].set_title(title, color='red')
            else:
                axs[j].imshow(pic, extent=[0, 20, 0, 15], aspect=1)
                axs[j].set_title(title)
                axs[j].axis('off')
                if types[list(predictions[j]).index(predictions[j].max())] != types[list(labels[j]).index(1)]:
                    axs[i,j].set_title(title, color='red')
    plt.tight_layout()
    plt.show()

In [12]:
def con_fu(rows, test_batches, model):
    """ 
    Collects data for confusion_matrix 
  
    Used in conjunction with sklearn.metrics confusion_matrix function
  
    Parameters: 
    rows (int): number of pictures will be predicted on
    test_batches (ImageDataGenerator) : the pictures that will be predicted
    model : the model training images have been trained on to be used to predict test images
    
    
    
    Returns: 
    actual (list) : list of actual class per image
    preds (list) : list of predicted class per image
    miss_class (list) : list of images that were missclassified
    """
    actual = []
    preds = []
    miss_class = []
    miss_predictions = []
    for i in range(rows):
        images, labels = next(test_batches)
        predictions = model.predict(images)
        for j, h in enumerate(labels):
            act_ = list(h).index(1.)
            pred_ = list(predictions[j]).index(predictions[j].max())
            actual.append(act_)
            preds.append(pred_)
        if act_ != pred_:
            miss_class.append(images[j])
            miss_predictions.append([act_, pred_])
    return actual, preds, miss_class, miss_predictions

In [13]:
# To Make image plots we need to read in images with a smaller batch size
test_batches2 = ImageDataGenerator(rescale=1/255).flow_from_directory(test_path, target_size=(224, 224), batch_size=4)

Found 1148 images belonging to 10 classes.


In [15]:
# NEED TO SAVE THE MODEL FROM GOOGLE DRIVE
# Use helper function to plot correct/incorrect predictions
image_plot_predict(4, test_batches2, resn_model, cols=4)

NameError: name 'resn_model' is not defined